In [1]:
pip install gmaps

In [2]:
conda install -c conda-forge gmaps

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\norma\anaconda3\envs\PythonData

  added / updated specs:
    - gmaps


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.6.15  |       h5b45459_0         188 KB  conda-forge
    certifi-2022.6.15          |   py37h03978a9_0         156 KB  conda-forge
    openssl-1.1.1q             |       h8ffe710_0         5.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         6.1 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.7-2_cp37m

The following packages will be UPDATED:

  conda              pkgs/main::conda-4.13.0-py37haa95532_0 --> conda-forge::conda-4.13.0-py37h03978a9_1

The following packages will be SUPERSEDED by a higher-priority cha

In [3]:
# Import dependencies
import gmaps
import requests
import pandas as pd

# import the API key
from config import g_key


In [4]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,57.27,77,98,22.41,AU,2022-08-07 22:20:03
1,1,Yellowknife,62.4560,-114.3525,63.12,67,75,4.61,CA,2022-08-07 22:17:01
2,2,Upernavik,72.7868,-56.1549,42.46,83,84,8.30,GL,2022-08-07 22:20:04
3,3,Saint Anthony,45.0205,-93.2180,75.25,73,100,9.22,US,2022-08-07 22:15:29
4,4,Najran,17.4924,44.1277,83.05,44,9,7.18,SA,2022-08-07 22:20:04


In [5]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [6]:
# configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

In [9]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 65


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,57.27,77,98,22.41,AU,2022-08-07 22:20:03
1,1,Yellowknife,62.4560,-114.3525,63.12,67,75,4.61,CA,2022-08-07 22:17:01
9,9,Carnarvon,-24.8667,113.6333,61.75,79,0,11.25,AU,2022-08-07 22:20:06
26,26,Carlos Antonio Lopez,-26.4000,-54.7667,60.69,98,100,1.54,PY,2022-08-07 22:20:10
27,27,Castro,-24.7911,-50.0119,57.47,95,100,4.07,BR,2022-08-07 22:20:10
29,29,Ahipara,-35.1667,173.1667,58.24,85,12,7.67,NZ,2022-08-07 22:20:11
31,31,Lubango,-14.9172,13.4925,61.97,23,0,4.21,AO,2022-08-07 22:20:11
33,33,Gacko,43.1672,18.5353,60.19,90,0,5.68,BA,2022-08-07 22:20:12
35,35,Kodiak,57.7900,-152.4072,60.31,59,20,5.75,US,2022-08-07 22:20:12
37,37,Havre-Saint-Pierre,50.2334,-63.5986,62.96,82,0,13.80,CA,2022-08-07 22:20:10


In [13]:
preferred_cities_df.count()

City_ID       113
City          113
Lat           113
Lng           113
Max Temp      113
Humidity      113
Cloudiness    113
Wind Speed    113
Country       111
Date          113
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Busselton,AU,57.27,-33.6500,115.3333,
1,Yellowknife,CA,63.12,62.4560,-114.3525,
9,Carnarvon,AU,61.75,-24.8667,113.6333,
26,Carlos Antonio Lopez,PY,60.69,-26.4000,-54.7667,
27,Castro,BR,57.47,-24.7911,-50.0119,
29,Ahipara,NZ,58.24,-35.1667,173.1667,
31,Lubango,AO,61.97,-14.9172,13.4925,
33,Gacko,BA,60.19,43.1672,18.5353,
35,Kodiak,US,60.31,57.7900,-152.4072,
37,Havre-Saint-Pierre,CA,62.96,50.2334,-63.5986,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel Not Found... skipping")
        print(hotel_df)

Hotel Not Found... skipping
                     City Country  Max Temp      Lat       Lng  \
0               Busselton      AU     57.27 -33.6500  115.3333   
1             Yellowknife      CA     63.12  62.4560 -114.3525   
9               Carnarvon      AU     61.75 -24.8667  113.6333   
26   Carlos Antonio Lopez      PY     60.69 -26.4000  -54.7667   
27                 Castro      BR     57.47 -24.7911  -50.0119   
..                    ...     ...       ...      ...       ...   
566                  Okha      RU     55.06  53.5739  142.9478   
571                Bantry      IE     57.78  51.6833   -9.4500   
572             Tilichiki      RU     61.68  60.4667  166.1000   
581                 Mysen      NO     58.69  59.5535   11.3258   
591                Talara      PE     64.13  -4.5772  -81.2719   

                  Hotel Name  
0    Observatory Guest House  
1         The Explorer Hotel  
9      Hospitality Carnarvon  
26                  Milenium  
27            CHACARA BA

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [17]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [19]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))